<h3> Importy </h3>

In [22]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import csv

<h3> Przygotowanie danych </h3>
- Ustawienia urządzenia </br>
- Konwersja obrazu na znormalizowany tensor PyTorch 
&& normalizacja danych do wartości z zakresu [-1.0; 1.0] </br>
- Pobranie danych treningowych i testowych

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

<h3> Implementacja funkcji pomocniczych wykorzystywanych w zadaniu </h3>
- <b>Tworzenie podzbioru danych</b>, <i>zawierającego określony procent <b>losowych</b> danych (permutacja).</i></br>
- <b>Dodanie szumu Gaussa.</b> <i>Szum gaussowski to losowe zakłócenie o rozkładzie normalnym, stosowane w celu zwiększenia odporności modelu na zakłócenia w danych wejściowych</i> </br>


In [6]:
def get_subset(dataset, percentage):
    subset_size = int(len(dataset) * percentage)
    indices = torch.randperm(len(dataset))[:subset_size]
    return torch.utils.data.Subset(dataset, indices)

def add_gaussian_noise(data, std_dev):
    noise = torch.normal(0, std_dev, size=data.shape).to(data.device)
    return data + noise


<h3> Definiowanie modelu prostej sieci neuronowej </h3>

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(SimpleNN, self).__init__()
        layers = []
        sizes = [input_size] + hidden_sizes + [num_classes]
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1]))
            if i < len(sizes) - 2:
                layers.append(nn.ReLU())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


<h3> Trenowanie modelu </h3>

1. **Trening modelu:**
   - Przechodzi przez dane treningowe w wielu epokach.
   - Dla każdej epoki oblicza stratę (Loss) i aktualizuje wagi.
   - <i>Dodanie szumu do danych wejściowych (opcjonalne).</i>

2. **Ocena modelu:**
   - Po każdej epoce model jest oceniany na danych testowych.
   - Obliczana jest dokładność (Accuracy), która wskazuje, jak dobrze model radzi sobie na nowych danych, które nie były używane do trenowania.

3. **Wyniki:**
   - Funkcja zwraca dwie listy: jedną z wartościami strat dla treningu oraz drugą z dokładnością na danych testowych.


In [9]:
def train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, epochs, noise_std=0.0):
    model = model.to(device)
    train_losses, test_accuracies = [], []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.view(inputs.size(0), -1).to(device), labels.to(device)
            if noise_std > 0:
                inputs = add_gaussian_noise(inputs, noise_std)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        train_losses.append(epoch_loss / len(train_loader))

        # Ewaluacja
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.view(inputs.size(0), -1).to(device), labels.to(device)
                if noise_std > 0:
                    inputs = add_gaussian_noise(inputs, noise_std)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        test_accuracies.append(accuracy)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_losses[-1]:.4f}, Accuracy: {accuracy:.2f}%")

    return train_losses, test_accuracies

<h3> Przeprowadzenie eksperymentów

In [ ]:
input_size = 28 * 28
num_classes = 10
batch_sizes = [16, 64]
hidden_layer_sizes = [[128], [128, 64]]
train_sizes = [0.01, 0.1]
noise_stds = [0.0, 0.1, 0.2]
epochs = 10
learning_rate = 0.01

results = {}
criterion = nn.CrossEntropyLoss()

for train_size in train_sizes:
    for batch_size in batch_sizes:
        for hidden_sizes in hidden_layer_sizes:
            for noise_std in noise_stds:
                train_loader = torch.utils.data.DataLoader(get_subset(train_data, train_size), batch_size=batch_size, shuffle=True)
                test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
                model = SimpleNN(input_size, hidden_sizes, num_classes)
                optimizer = optim.SGD(model.parameters(), lr=learning_rate)
                print(f"\nTrain size: {train_size*100}%, Batch size: {batch_size}, Hidden sizes: {hidden_sizes}, Noise std: {noise_std}")
                losses, accuracies = train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, epochs, noise_std)
                results[(train_size, batch_size, tuple(hidden_sizes), noise_std)] = (losses, accuracies)



Train size: 1.0%, Batch size: 16, Hidden sizes: [128], Noise std: 0.0
Epoch 1/10, Loss: 2.0224, Accuracy: 52.17%
Epoch 2/10, Loss: 1.5234, Accuracy: 65.79%
Epoch 3/10, Loss: 1.2127, Accuracy: 63.48%
Epoch 4/10, Loss: 1.0375, Accuracy: 68.50%
Epoch 5/10, Loss: 0.9121, Accuracy: 70.70%
Epoch 6/10, Loss: 0.8254, Accuracy: 71.76%
Epoch 7/10, Loss: 0.7647, Accuracy: 72.50%
Epoch 8/10, Loss: 0.7249, Accuracy: 71.19%
Epoch 9/10, Loss: 0.6828, Accuracy: 73.49%
Epoch 10/10, Loss: 0.6564, Accuracy: 74.14%

Train size: 1.0%, Batch size: 16, Hidden sizes: [128], Noise std: 0.1
Epoch 1/10, Loss: 2.0668, Accuracy: 45.83%
Epoch 2/10, Loss: 1.5955, Accuracy: 58.28%
Epoch 3/10, Loss: 1.2826, Accuracy: 62.48%
Epoch 4/10, Loss: 1.0868, Accuracy: 64.85%
Epoch 5/10, Loss: 0.9804, Accuracy: 68.45%
Epoch 6/10, Loss: 0.8873, Accuracy: 70.97%
Epoch 7/10, Loss: 0.8340, Accuracy: 69.87%
Epoch 8/10, Loss: 0.7860, Accuracy: 72.21%
Epoch 9/10, Loss: 0.7448, Accuracy: 72.79%
Epoch 10/10, Loss: 0.7188, Accuracy: 70.

<h3>Wizualizacja wyników na wykresach & zapis do pliku .csv</h3>

Results saved to results.csv


In [24]:
def plot_results(results):
    plt.figure(figsize=(15, 15))

    plot_index = 1
    for noise_std in noise_stds:
        for batch_size in batch_sizes:
            filtered_losses = []
            filtered_accuracies = []
            labels = []
            for key, (losses, accuracies) in results.items():
                train_size, batch_size_key, hidden_sizes, noise_std_key = key
                if batch_size_key == batch_size and noise_std_key == noise_std:
                    filtered_losses.append(losses)
                    filtered_accuracies.append(accuracies)
                    labels.append(f"Train size: {train_size*100}%, Hidden sizes: {hidden_sizes}")

            # Loss
            plt.subplot(6, 2, plot_index)
            for losses, label in zip(filtered_losses, labels):
                plt.plot(losses, label=label)
            plt.title(f"Training Loss - Noise: {noise_std}, Batch: {batch_size}")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.legend()

            # Accuracy
            plot_index += 1
            plt.subplot(6, 2, plot_index)
            for accuracies, label in zip(filtered_accuracies, labels):
                plt.plot(accuracies, label=label)
            plt.title(f"Test Accuracy - Noise: {noise_std}, Batch: {batch_size}")
            plt.xlabel("Epoch")
            plt.ylabel("Accuracy")
            plt.legend()

            plot_index += 1

    plt.tight_layout()
    plt.show()

def save_results_to_csv(results, filename="results.csv"):
    headers = ['Train size', 'Batch size', 'Hidden sizes', 'Noise std', 'Epoch', 'Loss', 'Accuracy']
    
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for key, (losses, accuracies) in results.items():
            train_size, batch_size, hidden_sizes, noise_std = key
            for epoch in range(len(losses)):
                writer.writerow([train_size*100, batch_size, hidden_sizes, noise_std, epoch+1, losses[epoch], accuracies[epoch]])

    print(f"Results saved to {filename}")

save_results_to_csv(results)

plot_results(results)
